# Caso de Uso: Detección de fraude en consumo energético

El fraude energético es un problema significativo para las compañías de electricidad, ya que genera pérdidas económicas, afecta la estabilidad del sistema y puede derivar en riesgos de seguridad. Muchas de estas irregularidades ocurren cuando los clientes manipulan sus medidores de electricidad o reportan consumos menores a los reales para pagar facturas más bajas.

En este contexto, se requiere desarrollar un modelo de detección de fraude que permita identificar clientes con patrones de consumo anómalos y optimizar las inspecciones comerciales. Esto permitirá reducir costos operativos y mejorar la eficiencia en la identificación de fraudes.

## Objetivos

Objetivo del análisis El objetivo de este análisis es detectar clientes con alta probabilidad de fraude utilizando datos de consumo energético y características de facturación. Para ello, se analizarán patrones en los niveles de consumo, estados de los medidores y diferencias en las lecturas de los contadores. Se explorarán anomalías y se evaluará si ciertos comportamientos están asociados a casos de fraude.

Los pasos clave del análisis incluyen:
- Exploración de datos para entender patrones de consumo.
- Identificación de valores atípicos en el consumo y diferencias de medición.
- Análisis de fraude (target) y su relación con las variables disponibles.
- Desarrollo de un modelo predictivo para clasificar clientes fraudulentos.

## Descripción de los datasets

1. **Dataset "clients1_3.csv"**: Contiene información sobre 135,493 clientes con las siguientes variables:
- disrict: Código numérico del distrito (Integer)
- client_id: Identificador único del cliente (String)
- client_catg: Categoría del cliente (Integer)
- region: Código de región geográfica (Integer)
- creation_date: Fecha de alta del cliente (String, formato DD/MM/YYYY)
- target: Variable objetivo donde 1.0 indica fraude y 0.0 indica no fraude (Float)

2. **Dataset "invoices1.csv"**: Contiene 2,238,374 registros de facturación con información detallada:
- client_id: Identificador del cliente que permite vincular con el dataset anterior
- invoice_date: Fecha de emisión de la factura
- tarif_type: Tipo de tarifa aplicada
- counter_number: Número identificador del contador
- counter_statue: Estado del contador
- counter_code: Código del contador
- reading_remarque: Observaciones sobre la lectura
- counter_coefficient: Coeficiente utilizado para la medición
- consommation_level_1,2,3,4: Niveles de consumo en diferentes franjas
- old_index: Lectura anterior del contador
- new_index: Lectura actual del contador
- months_number: Número de meses incluidos en la factura
- counter_type: Tipo de contador (ej. "ELEC" para electricidad)

---

# Notebook 2: Preprocesamiento y Feature Engineering

## 1. Librerías

In [1]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
import warnings
warnings.filterwarnings('ignore')

## 2. Cargar datos

In [2]:
file_invoice = '/home/ngonzalez/mi_pagina_personal/inesdi_ml/invoices.csv'
file_cliente = '/home/ngonzalez/mi_pagina_personal/inesdi_ml/clients.csv'
invoices_df = pd.read_csv(file_invoice)
clients_df = pd.read_csv(file_cliente)

In [3]:
invoices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2238374 entries, 0 to 2238373
Data columns (total 17 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Unnamed: 0            int64 
 1   client_id             object
 2   invoice_date          object
 3   tarif_type            int64 
 4   counter_number        int64 
 5   counter_statue        object
 6   counter_code          int64 
 7   reading_remarque      int64 
 8   counter_coefficient   int64 
 9   consommation_level_1  int64 
 10  consommation_level_2  int64 
 11  consommation_level_3  int64 
 12  consommation_level_4  int64 
 13  old_index             int64 
 14  new_index             int64 
 15  months_number         int64 
 16  counter_type          object
dtypes: int64(13), object(4)
memory usage: 290.3+ MB


El dataset de invoices Contiene 2,238,374 registros y 17 columnas. La mayoría de las columnas son de tipo int64, excepto cuatro que son de tipo object: client_id, invoice_date, counter_statue y counter_type.

In [4]:
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   disrict        135493 non-null  int64  
 1   client_id      135493 non-null  object 
 2   client_catg    135493 non-null  int64  
 3   region         135493 non-null  int64  
 4   creation_date  135493 non-null  object 
 5   target         135493 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 6.2+ MB


El datatset de clientes Contiene 135,493 registros y 6 columnas. Mezcla tipos de datos int64, float64 y object.

In [5]:
# Verificación de las primeras filas
print("\nPrimeras filas del dataset de clientes:")
clients_df.head()


Primeras filas del dataset de clientes:


,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [6]:
print("\nPrimeras filas del dataset de facturas:")
invoices_df.head()


Primeras filas del dataset de facturas:


,Unnamed: 0,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,1797067,train_Client_27015,2009-04-20,15,4051646,0,202,6,1,0,0,0,0,25853,25853,4,ELEC
1,1314893,train_Client_13786,2013-02-04,40,6767497,0,5,8,1,54,0,0,0,2432,2486,4,GAZ
2,4234799,train_Client_93401,2008-11-20,40,6849486,0,5,6,1,0,0,0,0,0,0,4,GAZ
3,2779039,train_Client_53889,2019-09-01,11,622828,0,413,9,1,1600,633,0,0,11033,13266,8,ELEC
4,3484865,train_Client_73134,2018-08-24,11,2165708268000,0,203,9,1,800,400,480,0,2672,4352,4,ELEC


## 3. Pre-Procesado

### Tipo de Variables

In [7]:
# Verificación de tipos de datos
print("\nTipos de datos en el dataset de clientes:")
print(clients_df.dtypes)

print("\nTipos de datos en el dataset de facturas:")
print(invoices_df.dtypes)


Tipos de datos en el dataset de clientes:
disrict            int64
client_id         object
client_catg        int64
region             int64
creation_date     object
target           float64
dtype: object

Tipos de datos en el dataset de facturas:
Unnamed: 0               int64
client_id               object
invoice_date            object
tarif_type               int64
counter_number           int64
counter_statue          object
counter_code             int64
reading_remarque         int64
counter_coefficient      int64
consommation_level_1     int64
consommation_level_2     int64
consommation_level_3     int64
consommation_level_4     int64
old_index                int64
new_index                int64
months_number            int64
counter_type            object
dtype: object


### Conversión de variables categóricas

In [8]:
# Función para detectar si una variable numérica es probablemente categórica
def es_probablemente_categorica(serie):
    """
    Determina si una variable numérica es probablemente categórica basándose en:
    1. Número de valores únicos (cardinalidad)
    2. Proporción de valores únicos respecto al total
    3. Si todos los valores son enteros
    """
    # Excluir nulos
    serie = serie.dropna()
    
    # Si está vacía, no podemos determinar
    if len(serie) == 0:
        return False
    
    # Número de valores únicos
    valores_unicos = serie.nunique()
    
    # Proporción de valores únicos
    proporcion_unicos = valores_unicos / len(serie)
    
    # Verificar si todos los valores son enteros
    todos_enteros = True
    for x in serie.sample(min(1000, len(serie))):
        if pd.notna(x):  # Verificar que no sea NaN
            todos_enteros = todos_enteros and float(x).is_integer()
    
    # Criterios para considerar una variable como categórica
    if todos_enteros and (valores_unicos <= 20 or proporcion_unicos < 0.05):
        return True
    else:
        return False

In [9]:
# Analizar variables numéricas en ambos datasets
print("\nVariables potencialmente categóricas en el dataset de clientes:")
for col in clients_df.select_dtypes(include=['number']).columns:
    if es_probablemente_categorica(clients_df[col]):
        print(f"- {col}: {clients_df[col].nunique()} valores únicos")
        print(f"  Valores: {sorted(clients_df[col].unique())}")


Variables potencialmente categóricas en el dataset de clientes:
- disrict: 4 valores únicos
  Valores: [60, 62, 63, 69]
- client_catg: 3 valores únicos
  Valores: [11, 12, 51]
- region: 25 valores únicos
  Valores: [101, 103, 104, 105, 106, 107, 199, 206, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 371, 372, 379, 399]
- target: 2 valores únicos
  Valores: [0.0, 1.0]


In [10]:
print("\nVariables potencialmente categóricas en el dataset de facturas:")
for col in invoices_df.select_dtypes(include=['number']).columns:
    if es_probablemente_categorica(invoices_df[col]):
        print(f"- {col}: {invoices_df[col].nunique()} valores únicos")
        print(f"  Valores: {sorted(invoices_df[col].unique())[:10]}{'...' if invoices_df[col].nunique() > 10 else ''}")


Variables potencialmente categóricas en el dataset de facturas:
- tarif_type: 17 valores únicos
  Valores: [8, 9, 10, 11, 12, 13, 14, 15, 18, 21]...
- counter_code: 41 valores únicos
  Valores: [0, 5, 10, 16, 25, 40, 65, 101, 102, 201]...
- reading_remarque: 7 valores únicos
  Valores: [6, 7, 8, 9, 203, 207, 413]
- counter_coefficient: 14 valores únicos
  Valores: [0, 1, 2, 3, 4, 6, 8, 9, 10, 20]...
- consommation_level_1: 6535 valores únicos
  Valores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
- consommation_level_2: 9836 valores únicos
  Valores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
- consommation_level_3: 1843 valores únicos
  Valores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
- consommation_level_4: 9126 valores únicos
  Valores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
- months_number: 760 valores únicos
  Valores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...


In [11]:
# Convertir variables numéricas que son categóricas
print("\nConvirtiendo variables numéricas categóricas a tipo categórico:")

# Lista de variables a convertir en dataset de clientes
cat_cols_clients = ['disrict', 'client_catg', 'region', 'target']
for col in cat_cols_clients:
    if col in clients_df.columns:
        clients_df[col] = clients_df[col].astype('category')
        print(f"- {col} convertida a tipo categórico")

# Lista de variables a convertir en dataset de facturas
cat_cols_invoices = ['tarif_type', 'counter_statue', 'counter_code', 'reading_remarque']
for col in cat_cols_invoices:
    if col in invoices_df.columns:
        invoices_df[col] = invoices_df[col].astype('category')
        print(f"- {col} convertida a tipo categórico")


Convirtiendo variables numéricas categóricas a tipo categórico:
- disrict convertida a tipo categórico
- client_catg convertida a tipo categórico
- region convertida a tipo categórico
- target convertida a tipo categórico
- tarif_type convertida a tipo categórico
- counter_statue convertida a tipo categórico
- counter_code convertida a tipo categórico
- reading_remarque convertida a tipo categórico


### Valores Nulos

In [12]:
# Verificación de valores nulos
print("\nValores nulos en el dataset de clientes:")
null_clients = clients_df.isnull().sum()
print(null_clients[null_clients > 0] if null_clients.any() > 0 else "No hay valores nulos")

print("\nValores nulos en el dataset de facturas:")
null_invoices = invoices_df.isnull().sum()
print(null_invoices[null_invoices > 0] if null_invoices.any() > 0 else "No hay valores nulos")


Valores nulos en el dataset de clientes:
No hay valores nulos

Valores nulos en el dataset de facturas:
No hay valores nulos


### Conversión de fechas

In [13]:
print("\nConvirtiendo fechas a formato datetime:")
clients_df['creation_date'] = pd.to_datetime(clients_df['creation_date'], format='%d/%m/%Y', errors='coerce')
invoices_df['invoice_date'] = pd.to_datetime(invoices_df['invoice_date'], errors='coerce')

# Verificar conversión
print(f"- creation_date tiene {clients_df['creation_date'].isnull().sum()} valores nulos después de conversión")
print(f"- invoice_date tiene {invoices_df['invoice_date'].isnull().sum()} valores nulos después de conversión")


Convirtiendo fechas a formato datetime:
- creation_date tiene 0 valores nulos después de conversión
- invoice_date tiene 0 valores nulos después de conversión


### Tratamiento de Outliers

In [14]:
# Variables de consumo para analizar outliers
consumo_cols = ['consommation_level_1', 'consommation_level_2', 'consommation_level_3', 
                'consommation_level_4', 'old_index', 'new_index']

# Análisis de outliers sin tratamiento
print("Análisis de outliers sin imputación ni recorte:")
for col in consumo_cols:
    if col in invoices_df.columns:
        # Calcular estadísticas descriptivas
        stats = invoices_df[col].describe()
        
        # Calcular IQR y límites
        q1 = stats['25%']
        q3 = stats['75%']
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        # Identificar outliers sin modificarlos
        outliers_lower = (invoices_df[col] < lower_bound)
        outliers_upper = (invoices_df[col] > upper_bound)
        total_outliers_lower = outliers_lower.sum()
        total_outliers_upper = outliers_upper.sum()
        total_outliers = total_outliers_lower + total_outliers_upper
        
        # Calcular porcentajes
        count_no_null = invoices_df[col].count()
        pct_outliers_lower = (total_outliers_lower / count_no_null) * 100
        pct_outliers_upper = (total_outliers_upper / count_no_null) * 100
        pct_outliers_total = (total_outliers / count_no_null) * 100
        
        # Valores extremos
        min_val = invoices_df[col].min()
        max_val = invoices_df[col].max()
        
        # Reportar resultados
        print(f"\n{col}:")
        print(f"- Estadísticas básicas: min={min_val}, q1={q1}, mediana={stats['50%']}, q3={q3}, max={max_val}")
        print(f"- Rango intercuartílico (IQR): {iqr}")
        print(f"- Límites para detección de outliers: [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"- Outliers por debajo del límite: {total_outliers_lower} ({pct_outliers_lower:.2f}%)")
        print(f"- Outliers por encima del límite: {total_outliers_upper} ({pct_outliers_upper:.2f}%)")
        print(f"- Total outliers: {total_outliers} ({pct_outliers_total:.2f}%)")
        
        # Para niveles 2-4, mostrar información adicional si hay muchos ceros
        if 'consommation_level' in col and col != 'consommation_level_1':
            zeros = (invoices_df[col] == 0).sum()
            pct_zeros = (zeros / count_no_null) * 100
            print(f"- Valores cero: {zeros} ({pct_zeros:.2f}% del total)")
            
            # Si hay muchos ceros, mostrar estadísticas para valores no cero
            if pct_zeros > 50:
                no_zeros = invoices_df[invoices_df[col] > 0][col]
                if len(no_zeros) > 0:
                    print(f"- Estadísticas para valores no cero (n={len(no_zeros)}):")
                    print(f"  min={no_zeros.min()}, mediana={no_zeros.median()}, max={no_zeros.max()}")

Análisis de outliers sin imputación ni recorte:

consommation_level_1:
- Estadísticas básicas: min=0, q1=79.0, mediana=274.0, q3=600.0, max=126118
- Rango intercuartílico (IQR): 521.0
- Límites para detección de outliers: [-702.50, 1381.50]
- Outliers por debajo del límite: 0 (0.00%)
- Outliers por encima del límite: 62192 (2.78%)
- Total outliers: 62192 (2.78%)

consommation_level_2:
- Estadísticas básicas: min=0, q1=0.0, mediana=0.0, q3=0.0, max=819886
- Rango intercuartílico (IQR): 0.0
- Límites para detección de outliers: [0.00, 0.00]
- Outliers por debajo del límite: 0 (0.00%)
- Outliers por encima del límite: 330860 (14.78%)
- Total outliers: 330860 (14.78%)
- Valores cero: 1907514 (85.22% del total)
- Estadísticas para valores no cero (n=330860):
  min=1, mediana=369.0, max=819886

consommation_level_3:
- Estadísticas básicas: min=0, q1=0.0, mediana=0.0, q3=0.0, max=36273
- Rango intercuartílico (IQR): 0.0
- Límites para detección de outliers: [0.00, 0.00]
- Outliers por debajo 

In [15]:
# Calcular diferencia entre índices
invoices_df['diff_index'] = invoices_df['new_index'] - invoices_df['old_index']

# Identificar diferencias extremas
q3_diff = invoices_df['diff_index'].quantile(0.75)
upper_bound_diff = q3_diff * 3
invoices_df['extreme_diff'] = (invoices_df['diff_index'] > upper_bound_diff).astype(int)

print(f"\nFacturas con diferencia extremadamente alta entre índices: {invoices_df['extreme_diff'].sum()} ({invoices_df['extreme_diff'].mean()*100:.2f}%)")



Facturas con diferencia extremadamente alta entre índices: 109855 (4.91%)


## 4. Feature Engineering

In [ ]:
# 1. PREPARACIÓN INICIAL
# Calcular consumo total para cada factura
invoices_df['consumo_total'] = (
    invoices_df['consommation_level_1'].fillna(0) + 
    invoices_df['consommation_level_2'].fillna(0) + 
    invoices_df['consommation_level_3'].fillna(0) + 
    invoices_df['consommation_level_4'].fillna(0)
)

# Calcular diferencia entre índices
invoices_df['diff_indices'] = invoices_df['new_index'] - invoices_df['old_index']

In [17]:
# 2. AGREGACIÓN DE FACTURAS POR CLIENTE
# Definir funciones de agregación
agg_functions = {
    'invoice_date': ['count'],
    'months_number': ['sum'],
    'consumo_total': ['sum', 'mean', 'std'],
    'diff_indices': ['mean', 'std'],
    'old_index': ['min', 'max'],
    'new_index': ['min', 'max']
}

# Agregar por cliente
fact_por_cliente = invoices_df.groupby('client_id').agg(agg_functions)

# Aplanar columnas multindice
fact_por_cliente.columns = ['_'.join(col).strip('_') for col in fact_por_cliente.columns]

# Renombrar para claridad
fact_por_cliente = fact_por_cliente.rename(columns={
    'invoice_date_count': 'total_facturas',
    'months_number_sum': 'meses_totales',
    'consumo_total_sum': 'consumo_total',
    'consumo_total_mean': 'consumo_promedio',
    'consumo_total_std': 'variabilidad_consumo',
    'diff_indices_mean': 'promedio_diferencia_indices'
})

# Calcular consumo mensual
fact_por_cliente['consumo_mensual'] = fact_por_cliente['consumo_total'] / fact_por_cliente['meses_totales']
fact_por_cliente['consumo_mensual'] = fact_por_cliente['consumo_mensual'].replace([np.inf, -np.inf], np.nan).fillna(0)

In [18]:
# 3. VARIABLES TEMPORALES
# Calcular antigüedad
reference_date = pd.Timestamp('2020-01-01')
clients_df['antiguedad_anios'] = (reference_date - clients_df['creation_date']).dt.days / 365.25

# Extraer año de creación
clients_df['creation_year'] = clients_df['creation_date'].dt.year

# Crear cohortes temporales
year_bins = [1975, 1985, 1995, 2005, 2015, 2025]
clients_df['cohorte'] = pd.cut(clients_df['creation_year'], bins=year_bins, 
                             labels=['1975-1984', '1985-1994', '1995-2004', '2005-2014', '2015+'])

In [19]:
# 4. DETECCIÓN DE INCONSISTENCIAS TÉCNICAS PARA TODOS LOS CLIENTES
print("Detectando inconsistencias técnicas para todos los clientes...")
print("(Este proceso puede tomar tiempo)")

# Definir función para detectar inconsistencias
def detectar_inconsistencias_tecnicas(facturas_cliente):
    if len(facturas_cliente) <= 1:
        return {
            'tiene_inconsistencias_tecnicas': 0,
            'indices_negativos': 0,
            'indices_cero': 0
        }
    
    # Ordenar por fecha
    facturas_cliente = facturas_cliente.sort_values('invoice_date')
    
    # Detectar inconsistencias
    indices_negativos = (facturas_cliente['diff_indices'] < 0).any()
    indices_cero = ((facturas_cliente['diff_indices'] == 0) & 
                   (facturas_cliente['consumo_total'] > 0)).any()
    
    return {
        'tiene_inconsistencias_tecnicas': int(indices_negativos or indices_cero),
        'indices_negativos': int(indices_negativos),
        'indices_cero': int(indices_cero)
    }

# Obtener todos los client_id únicos de las facturas
all_clients = invoices_df['client_id'].unique()
total_clients = len(all_clients)

# Detectar inconsistencias para cada cliente
inconsistencias_dict = {}
for i, client_id in enumerate(all_clients):
    # Mostrar progreso cada 1000 clientes para no saturar la consola
    if i % 1000 == 0 or i == total_clients - 1:
        print(f"Procesando cliente {i+1}/{total_clients} ({((i+1)/total_clients*100):.1f}%)...")
    
    facturas_cliente = invoices_df[invoices_df['client_id'] == client_id]
    if len(facturas_cliente) > 0:
        inconsistencias_dict[client_id] = detectar_inconsistencias_tecnicas(facturas_cliente)

# Convertir a DataFrame
inconsistencias_df = pd.DataFrame.from_dict(inconsistencias_dict, orient='index')

# Unir con facturas por cliente
fact_por_cliente = fact_por_cliente.join(inconsistencias_df, how='left')

# Verificar si hay valores nulos después de la unión
nulos = fact_por_cliente[['tiene_inconsistencias_tecnicas', 'indices_negativos', 'indices_cero']].isnull().sum()
print("\nValores nulos en las columnas de inconsistencias después de la unión:")
print(nulos)

# Completar valores nulos (debería haber pocos o ninguno)
for col in ['tiene_inconsistencias_tecnicas', 'indices_negativos', 'indices_cero']:
    if col in fact_por_cliente.columns and fact_por_cliente[col].isnull().sum() > 0:
        fact_por_cliente[col] = fact_por_cliente[col].fillna(0)
        print(f"- Completados {fact_por_cliente[col].isnull().sum()} valores nulos en {col}")

# Mostrar estadísticas sobre las inconsistencias
total_con_inconsistencias = fact_por_cliente['tiene_inconsistencias_tecnicas'].sum()
porcentaje = (total_con_inconsistencias / len(fact_por_cliente)) * 100

print(f"\nEstadísticas finales de inconsistencias:")
print(f"- Total de clientes analizados: {len(fact_por_cliente)}")
print(f"- Clientes con inconsistencias técnicas: {total_con_inconsistencias} ({porcentaje:.2f}%)")
print(f"- Clientes con índices negativos: {fact_por_cliente['indices_negativos'].sum()}")
print(f"- Clientes con índices cero pero consumo: {fact_por_cliente['indices_cero'].sum()}")

Detectando inconsistencias técnicas para todos los clientes...
(Este proceso puede tomar tiempo)
Procesando cliente 1/130867 (0.0%)...
Procesando cliente 1001/130867 (0.8%)...
Procesando cliente 2001/130867 (1.5%)...
Procesando cliente 3001/130867 (2.3%)...
Procesando cliente 4001/130867 (3.1%)...
Procesando cliente 5001/130867 (3.8%)...
Procesando cliente 6001/130867 (4.6%)...
Procesando cliente 7001/130867 (5.3%)...
Procesando cliente 8001/130867 (6.1%)...
Procesando cliente 9001/130867 (6.9%)...
Procesando cliente 10001/130867 (7.6%)...
Procesando cliente 11001/130867 (8.4%)...
Procesando cliente 12001/130867 (9.2%)...
Procesando cliente 13001/130867 (9.9%)...
Procesando cliente 14001/130867 (10.7%)...
Procesando cliente 15001/130867 (11.5%)...
Procesando cliente 16001/130867 (12.2%)...
Procesando cliente 17001/130867 (13.0%)...
Procesando cliente 18001/130867 (13.8%)...
Procesando cliente 19001/130867 (14.5%)...
Procesando cliente 20001/130867 (15.3%)...
Procesando cliente 21001/13

In [21]:
# 5. VARIABLES GEOGRÁFICAS CONTEXTUALIZADAS
# Convertir target a tipo numérico si está como categórica
if clients_df['target'].dtype.name == 'category':
    clients_df['target'] = clients_df['target'].astype(int)

categorical_cols_clients = ['disrict', 'client_catg', 'region']  # Elimina 'target' de esta lista
for col in categorical_cols_clients:
    clients_df[col] = clients_df[col].astype('category')

# Calcular tasa base de fraude
fraude_base = clients_df['target'].mean()
print(f"Tasa de fraude global: {fraude_base:.4f}")

# Calcular tasas por región con regularización bayesiana
region_counts = clients_df.groupby('region').size()
region_fraud = clients_df.groupby('region')['target'].mean()
alpha = 10  # Factor de regularización
region_fraud_reg = (region_counts * region_fraud + alpha * fraude_base) / (region_counts + alpha)

# Crear DataFrame para unir después
region_df = pd.DataFrame({
    'region': region_fraud_reg.index,
    'tasa_fraude_reg': region_fraud_reg.values
})

Tasa de fraude global: 0.0558


In [22]:
# 6. NORMALIZACIÓN CONTEXTUALIZADA DE CONSUMO
# Primero unir clientes con consumo mensual
clients_with_consumption = clients_df[['client_id', 'region', 'cohorte']].copy()
clients_with_consumption = clients_with_consumption.merge(
    fact_por_cliente[['consumo_mensual']].reset_index(),
    on='client_id',
    how='left'
)

# Definir función para calcular percentiles por grupo
def calcular_percentiles_por_grupo(df, grupo, variable):
    result = {}
    for nombre, grupo_df in df.groupby(grupo):
        grupo_df = grupo_df.dropna(subset=[variable])
        if len(grupo_df) > 10:  # Solo si hay suficientes datos
            percentiles = {}
            for p in [25, 50, 75, 90, 95]:
                percentiles[p] = grupo_df[variable].quantile(p/100)
            result[nombre] = percentiles
    return result

# Calcular percentiles por cohorte y región
percentiles_cohorte = calcular_percentiles_por_grupo(clients_with_consumption, 'cohorte', 'consumo_mensual')
percentiles_region = calcular_percentiles_por_grupo(clients_with_consumption, 'region', 'consumo_mensual')

# Definir función para asignar nivel de consumo
def asignar_nivel_consumo(consumo, grupo, percentiles_dict):
    if grupo not in percentiles_dict or pd.isna(consumo):
        return 'desconocido'
    
    percentiles = percentiles_dict[grupo]
    if consumo <= percentiles[25]:
        return 'bajo'
    elif consumo <= percentiles[75]:
        return 'normal'
    elif consumo <= percentiles[95]:
        return 'alto'
    else:
        return 'muy_alto'

# Aplicar a cada cliente
clients_with_consumption['nivel_consumo_cohorte'] = clients_with_consumption.apply(
    lambda x: asignar_nivel_consumo(x['consumo_mensual'], x['cohorte'], percentiles_cohorte), axis=1
)
clients_with_consumption['nivel_consumo_region'] = clients_with_consumption.apply(
    lambda x: asignar_nivel_consumo(x['consumo_mensual'], x['region'], percentiles_region), axis=1
)

## 5. Dataset Final 

In [23]:
# Unir clientes con facturas
dataset_final = clients_df.merge(
    fact_por_cliente.reset_index(), 
    left_on='client_id', 
    right_on='client_id', 
    how='left'
)

# Añadir tasa de fraude regularizada por región
dataset_final = dataset_final.merge(
    region_df, 
    on='region', 
    how='left'
)

# Añadir niveles de consumo contextualizados
dataset_final = dataset_final.merge(
    clients_with_consumption[['client_id', 'nivel_consumo_cohorte', 'nivel_consumo_region']],
    on='client_id',
    how='left'
)

# Manejar valores nulos
for col in dataset_final.columns:
    if col not in ['client_id', 'target', 'creation_date'] and dataset_final[col].isnull().sum() > 0:
        if dataset_final[col].dtype in [np.float64, np.int64]:
            dataset_final[col] = dataset_final[col].fillna(0)
        elif dataset_final[col].dtype == 'category' or pd.api.types.is_object_dtype(dataset_final[col]):
            dataset_final[col] = dataset_final[col].fillna('desconocido')

print(f"Dataset final creado con {dataset_final.shape[0]} filas y {dataset_final.shape[1]} columnas")

Dataset final creado con 135493 filas y 27 columnas


In [24]:
dataset_final.head()

,disrict,client_id,client_catg,region,creation_date,target,antiguedad_anios,creation_year,cohorte,total_facturas,...,old_index_max,new_index_min,new_index_max,consumo_mensual,tiene_inconsistencias_tecnicas,indices_negativos,indices_cero,tasa_fraude_reg,nivel_consumo_cohorte,nivel_consumo_region
0,60,train_Client_0,11,101,1994-12-31,0,25.002053,1994,1985-1994,17.0,...,15066.0,3950.0,15638.0,94.986842,0.0,0.0,0.0,0.035925,normal,normal
1,69,train_Client_1,11,107,2002-05-29,0,17.593429,2002,1995-2004,20.0,...,23264.0,5747.0,23590.0,126.097561,0.0,0.0,0.0,0.065803,normal,normal
2,62,train_Client_10,11,301,1986-03-13,0,33.804244,1986,1985-1994,8.0,...,41532.0,29497.0,44614.0,141.571429,0.0,0.0,0.0,0.033029,alto,alto
3,69,train_Client_100,11,105,1996-07-11,0,23.474333,1996,1995-2004,10.0,...,99.0,90.0,99.0,0.190476,0.0,0.0,0.0,0.055927,bajo,bajo
4,62,train_Client_1000,11,303,2014-10-14,0,5.215606,2014,2005-2014,8.0,...,13337.0,1624.0,13729.0,247.100000,0.0,0.0,0.0,0.054499,alto,alto


In [25]:
dataset_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   disrict                         135493 non-null  category      
 1   client_id                       135493 non-null  object        
 2   client_catg                     135493 non-null  category      
 3   region                          135493 non-null  category      
 4   creation_date                   135493 non-null  datetime64[ns]
 5   target                          135493 non-null  int64         
 6   antiguedad_anios                135493 non-null  float64       
 7   creation_year                   135493 non-null  int32         
 8   cohorte                         135493 non-null  category      
 9   total_facturas                  135493 non-null  float64       
 10  meses_totales                   135493 non-null  float64

### Guardar dataset

In [27]:
dataset_final.to_csv('/home/ngonzalez/mi_pagina_personal/inesdi_ml/dataset_final.csv', index=False)

## Conclusiones y próximos pasos

En este notebook hemos completado el preprocesamiento y la ingeniería de características para la detección de fraude energético, con especial atención a minimizar posibles sesgos y maximizar el poder predictivo:

**Preprocesamiento básico:**
- Identificación y conversión de variables categóricas
- Manejo de valores nulos
- Conversión adecuada de fechas

**Feature Engineering avanzado:**
- Variables temporales contextualizadas (cohortes)
- Variables geográficas con regularización bayesiana
- Detección de inconsistencias técnicas objetivas
- Normalización contextualizada del consumo

**Variables derivadas clave:**

- tasa_fraude_reg: Tasa regularizada por región
- nivel_consumo_cohorte/region: Categorización contextualizada del consumo
- tiene_inconsistencias_tecnicas: Detector de anomalías en lecturas
- consumo_mensual: Medida objetiva de consumo normalizado

**Dataset final preparado:**
- 135,493 registros
- 27 variables de alta calidad informativa
- Combinación de datos categóricos y numéricos


**Enfoque Metodológico**
El enfoque ha priorizado:
- Contextualización en lugar de etiquetado absoluto
- Regularización bayesiana para áreas con pocos datos
- Detección objetiva de anomalías técnicas
- Categorización multinivel del consumo

Este enfoque nos ha permitido crear un dataset robusto que captura tanto los aspectos técnicos como contextuales del fraude energético.

**Próximos Pasos**
Para completar el proyecto de detección de fraude, recomendamos:
1. Transformación Final de Variables
- Aplicar one-hot encoding a variables categóricas nominales (disrict, client_catg, region)
- Considerar encoding ordinal para variables con orden natural (nivel_consumo_cohorte, nivel_consumo_region)
- Analizar la distribución de variables numéricas y aplicar las transformaciones necesarias (normalización, escalado)

2. Modelado Predictivo
- Dividir los datos en conjuntos de entrenamiento, validación y prueba (70-15-15%)
- Implementar y comparar diferentes algoritmos:
    - Modelos lineales: Regresión Logística
    - Ensambles: Random Forest, Gradient Boosting, XGBoost
- Optimizar hiperparámetros mediante validación cruzada

3. Evaluación de Modelos
- Utilizar métricas apropiadas para detección de fraude: AUC-ROC, Precisión, Recall, F1-Score
- Análisis de costos de falsos positivos vs. falsos negativos
- Evaluar el rendimiento en diferentes segmentos para verificar equidad